In [1]:
from abc import ABCMeta, abstractmethod
from queue import PriorityQueue
import random
import math
import time

# Genetic Algorithms implementation

<img src="genetic.png" width="600"/>

In [2]:
class GeneticAlgorithm():
    """
    Genetic Algorithm implementation

    Methods
    -------
    evolve()
        Runs Genetic Algorithm, and returns the solution and the number of generations.
    get_random_state()
        Randomly generate a tuple describing a unique world configuration.
    recombine(state1, state2)
        Mix two states with the crossover operation.
    mutate(state)
        Randomly modify a given state.
    goal_test(state)
        Checks if the given state is a goal state.
    fitness_function(state)
        Checks how good the given state is.
    """
    __metaclass__ = ABCMeta
    def __init__(self, size, p):
        """
        Parameters
        ----------
        size
            An integer defining the population size.
        p
            Mutation probability
        """
        self.size = size
        self.p = p

    def evolve(self):
        """
        Runs Genetic Algorithm, and returns the solution and the number of generations.

        Returns
        -------
        tuple
            A tuple describing a unique world configuration.
        int
            The number of generations evaluated during the optimization.
        """

        # initial population
        population = [self.get_random_state() for _ in range(self.size)]
        generation = 1
        while True:
            fitness = []
            for state in population:
                if self.goal_test(state):
                    return state, generation
                fitness.append(1/self.fitness_function(state))
            max_fitness = max(fitness)

            # selection - spinning wheel without consecutive duplicates
            beta = 0
            index = int(random.random()*self.size)
            resampled_population = []
            last_index = -1
            for i in range(self.size):
                beta += random.random() * 2 * max_fitness
                while beta > fitness[index] or index == last_index:
                    if index != last_index:
                        beta -= fitness[index]
                    index = (index + 1)%self.size
                last_index = index
                resampled_population.append(population[index])

            # recombination using consecutive states
            for i in range(1, self.size, 2):
                resampled_population[i-1], resampled_population[i] = self.recombine(resampled_population[i-1], resampled_population[i])
            
            # mutate
            for i in range(self.size):
                if random.random() < self.p:
                    resampled_population[i] = self.mutate(resampled_population[i])

            population = resampled_population
            generation += 1

    @abstractmethod
    def get_random_state(self):
        """
        Randomly generate a tuple describing a unique world configuration.

        Returns
        -------
        tuple
            A tuple describing a unique world configuration.
        """
        pass
        
    @abstractmethod
    def recombine(self, state1, state2):
        """
        Mix two states with the crossover operation.

        Parameters
        ----------
        state1
            A tuple describing a unique world configuration.
        state2
            A tuple describing a unique world configuration.

        Returns
        -------
        tuple, tuple
            Two tuples describing unique world configurations.
        """
        pass

    @abstractmethod
    def mutate(self, state):
        """
        Randomly modify a given state.

        Parameters
        ----------
        state
            A tuple describing a unique world configuration.

        Returns
        -------
        tuple
            A tuple describing a unique world configuration.
        """
        pass

    @abstractmethod
    def goal_test(self, state):
        """
        Checks if the current state is a goal state.

        Parameters
        ----------
        state
            A tuple describing a unique world configuration.

        Returns
        -------
        bool
            True if the given state is a goal state, and False otherwise.
        """
        pass

    @abstractmethod
    def fitness_function(self, state):
        """
        Checks how good the given state is.

        Parameters
        ----------
        state
            A tuple describing a unique world configuration.

        Returns
        -------
        float
            The closer to the goal, the closer to 0 the returned value will be.
        """
        pass

# 8-queens

- Puzzle with chess queens in an $8\times8$ grid. The goal is to place 8 queens so that no queen attacks another.

<img src="8queens.png" width="300"/>

- **Heuristic function**: number of queen pairs attacking each other.

In [3]:
class KQueens(GeneticAlgorithm):
    """
    8-queens solution using GeneticAlgorithm.

    Methods
    -------
    show(state)
        Visualize the current state.
    get_random_state()
        Randomly generate a tuple describing a unique world configuration.
    recombine(state1, state2)
        Mix two states with the crossover operation.
    mutate(state)
        Randomly modify a given state.
    goal_test(state)
        Checks if the given state is a goal state.
    fitness_function(state)
        Checks how good the given state is.
    """

    def __init__(self, k=8, **kargs):
        """
        Parameters
        ----------
        k
            An integer defining the size of the board and the number of queens.
        """
        super().__init__(size=(kargs['size'] if 'size' in kargs else 10), p=(kargs['p'] if 'p' in kargs else 0.5))
        self.k = k

    def show(self, state):
        """
        Prints a given state.

        Parameters
        ----------
        state
            A tuple describing a unique world configuration.
        """
        print('╔'+'╦'.join(['═══']*self.k)+'╗')
        for i in range(self.k):
            print('║', end=' ')
            for j in range(self.k):
                if state[j] == i:
                    print('W', end=' ')
                else:
                    print(' ', end=' ')
                print('║', end=' ')
            print()
            if i < self.k-1:
                print('╠'+'╬'.join(['═══']*self.k)+'╣')
        print('╚'+'╩'.join(['═══']*self.k)+'╝')

    def get_random_state(self):
        """
        Randomly generate a tuple describing a unique world configuration.

        Returns
        -------
        tuple
            A tuple describing a unique world configuration.
        """
        return tuple(random.randrange(self.k) for i in range(self.k))
        
    def recombine(self, state1, state2):
        """
        Mix two states with the crossover operation.

        Parameters
        ----------
        state1
            A tuple describing a unique world configuration.
        state2
            A tuple describing a unique world configuration.

        Returns
        -------
        tuple, tuple
            Two tuples describing unique world configurations.
        """
        cross = random.randrange(self.k-1)+1
        state1, state2 = list(state1), list(state2)
        return tuple(state1[:cross]+state2[cross:]), tuple(state2[:cross]+state1[cross:])

    def mutate(self, state):
        """
        Randomly modify a given state.

        Parameters
        ----------
        state
            A tuple describing a unique world configuration.

        Returns
        -------
        tuple
            A tuple describing a unique world configuration.
        """
        state = list(state)
        state[random.randrange(self.k)] = random.randrange(self.k)
        return tuple(state)

    def fitness_function(self, state):
        """
        Checks how good the given state is.

        Parameters
        ----------
        state
            A tuple describing a unique world configuration.

        Returns
        -------
        float
            The closer to the goal, the closer to 0 the returned value will be.
        """
        cost = 0
        for i in range(self.k):
            for j in range(i+1,self.k):
                if state[i] == state[j] or j-i == abs(state[j]-state[i]):
                    cost += 1
        return cost

    def goal_test(self, state):
        """
        Checks if the current state is a goal state.

        Parameters
        ----------
        state
            A tuple describing a unique world configuration.

        Returns
        -------
        bool
            True if the given state is a goal state, and False otherwise.
        """
        return self.fitness_function(state) == 0

In [8]:
puzzle = KQueens(size=100, p=0.5)
state, num_generations = puzzle.evolve()
print('Result found in {} generations:'.format(num_generations))
puzzle.show(state)

Result found in 280 generations:
╔═══╦═══╦═══╦═══╦═══╦═══╦═══╦═══╗
║   ║   ║ W ║   ║   ║   ║   ║   ║ 
╠═══╬═══╬═══╬═══╬═══╬═══╬═══╬═══╣
║   ║   ║   ║   ║   ║   ║   ║ W ║ 
╠═══╬═══╬═══╬═══╬═══╬═══╬═══╬═══╣
║   ║   ║   ║ W ║   ║   ║   ║   ║ 
╠═══╬═══╬═══╬═══╬═══╬═══╬═══╬═══╣
║   ║   ║   ║   ║   ║   ║ W ║   ║ 
╠═══╬═══╬═══╬═══╬═══╬═══╬═══╬═══╣
║ W ║   ║   ║   ║   ║   ║   ║   ║ 
╠═══╬═══╬═══╬═══╬═══╬═══╬═══╬═══╣
║   ║   ║   ║   ║   ║ W ║   ║   ║ 
╠═══╬═══╬═══╬═══╬═══╬═══╬═══╬═══╣
║   ║ W ║   ║   ║   ║   ║   ║   ║ 
╠═══╬═══╬═══╬═══╬═══╬═══╬═══╬═══╣
║   ║   ║   ║   ║ W ║   ║   ║   ║ 
╚═══╩═══╩═══╩═══╩═══╩═══╩═══╩═══╝


In [9]:
for size in [6, 8, 10, 12, 14]:

    N = 100

    cost = 0

    start = time.time()
    for i in range(N):
        puzzle = KQueens(size=size, p=0.6)
        state, num_generations = puzzle.evolve()
        cost += num_generations
    stop = time.time()

    print('Population size: {}'.format(size))
    print('Average number of generations: {:.2f}'.format(cost/N))
    print('Average time: {:.2f}'.format((stop-start)/N))
    print()

Population size: 6
Average number of generations: 1388.11
Average time: 0.13

Population size: 8
Average number of generations: 480.92
Average time: 0.06

Population size: 10
Average number of generations: 301.91
Average time: 0.05

Population size: 12
Average number of generations: 240.41
Average time: 0.05

Population size: 14
Average number of generations: 203.45
Average time: 0.05



In [10]:
for p in [0.3, 0.4, 0.5, 0.6, 0.7]:

    N = 100

    cost = 0

    start = time.time()
    for i in range(N):
        puzzle = KQueens(size=10, p=p)
        state, num_generations = puzzle.evolve()
        cost += num_generations
    stop = time.time()

    print('Mutation rate: {}'.format(p))
    print('Average number of generations: {:.2f}'.format(cost/N))
    print('Average time: {:.2f}'.format((stop-start)/N))
    print()

Mutation rate: 0.3
Average number of generations: 942.33
Average time: 0.14

Mutation rate: 0.4
Average number of generations: 628.54
Average time: 0.10

Mutation rate: 0.5
Average number of generations: 324.80
Average time: 0.05

Mutation rate: 0.6
Average number of generations: 352.10
Average time: 0.06

Mutation rate: 0.7
Average number of generations: 468.98
Average time: 0.08

